In [1]:
import numpy as np
import cv2
from PIL import Image, ImageFilter

%matplotlib inline
import matplotlib.pyplot as plt

In [27]:
image_path = 'Sample-handwritten-text-input-for-OCR.png'
image_path = 'mal-hw.jpeg'
inp_img = Image.open(image_path)

In [51]:
# blurred = cv2.GaussianBlur(np.array(inp_img), (5,5), 0)
# blurred = inp_img.filter(ImageFilter.GaussianBlur(radius=2))
# blurred.show()
blurred = inp_img

In [52]:
def adaptive_thresholding(image: Image, block_size=3, offset=0):
    grayscale_img = image.convert('L')
    img_array = np.array(grayscale_img)
    # print(img_array.shape)

    height, width = img_array.shape

    #an empty array for the output
    thresholded_image = np.zeros((height, width))
    for y in range(0,height, block_size):
        for x in range(0, width, block_size):
            block = img_array[y:y+block_size, x:x+block_size]
            #find the mean
            block_mean = np.mean(block)
            # print(block_mean)
            thresholded_block = (block > (block_mean-offset)).astype(np.uint8)*255
            thresholded_image[y:y+block_size, x:x+block_size] = thresholded_block
    return Image.fromarray(thresholded_image)

In [53]:
out_image = adaptive_thresholding(blurred, 75, 70)
out_image.show(title='adaptive_thresholding')

In [54]:
def negate(image):
    img_arr = np.array(image)
    height, width = img_arr.shape
    for y in range(height):
        for x in range(width):
            img_arr[y,x] = (img_arr[y,x]==0).astype(np.uint8)*255
    return Image.fromarray(img_arr)

out_image = negate(out_image)
out_image.show()

In [55]:
# cv2.findContours(mask.copy, cv2.RETR_EXTERNAL, cv2.CHA )
# Find contours and get bounding box for each contour
cnts, _ = cv2.findContours(np.array(out_image,dtype=np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = [c for c in cnts if cv2.contourArea(c)>5]
boundingBoxes = [cv2.boundingRect(c) for c in cnts]

In [75]:
final = cv2.drawContours(np.array(inp_img, dtype=np.uint8),cnts, -1, (0,255,0), thickness=0, lineType=cv2.LINE_AA)
final = Image.fromarray(np.array(final))

In [71]:
final.show()

In [77]:
final=cv2.cvtColor(np.array(inp_img), cv2.COLOR_RGB2BGR)
for b in boundingBoxes:
    x,y,w,h = b
    final = cv2.rectangle(final, (x,y), (x+w,y+h), (0,255,0), 1)

In [78]:
# img = Imag
Image.fromarray(final).show()
# cv2.

In [79]:
character_images = []
for b in boundingBoxes:
    x, y, w, h = b
    character = final[y:y+h, x:x+w]
    character_images.append(character)

In [80]:
Image.fromarray(character_images[25]).show()